## Overall performance comparasion: 

Data: 55000 training data, 5000 validation data, 10000 test data; 
Step up: 5 layers with 100 node on each layer. 

Batch GD: waist a lot of time to converge. It needed 2000 step to have similar performance of mini-batch.

    time consumption: 1484.6108751296997
    performance score: 0.865
    
mini-batch GD: 

    time consumption: 89.35848474502563
    performance score: 0.9733
Adam: 
    
    time consumption:  114.90625596046448
    performance score:   0.1998

Adam with earily stoping: (35% threshold stop)

    time consumption: 51.90262746810913
    performance score: 0.9
   



In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
(X_data, y_data), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print (X_data.shape)
# reduce the dimention of X_train, X_test
def reduce_D (data):
    return data.astype (np.float32).reshape (-1, 28 * 28) / 255.0
X_data, X_test = reduce_D  (X_data), reduce_D (X_test)
print (X_data.shape, X_test.shape)

# Check Y :
# print (y_data.shape)
# def y_preprocess (data):
#     return data.astype (np.int32).reshape (data.shape[0], 1)
# y_data, y_test = y_preprocess (y_data), y_preprocess (y_test)
print (y_data.shape, y_test.shape)

# split the train and validation data:
X_validation, y_validation = X_data[:5000], y_data[:5000]
X_train, y_train = X_data[5000:], y_data[5000:]

print (X_validation.shape, X_train.shape, y_validation.shape)

(60000, 28, 28)
(60000, 784) (10000, 784)
(60000,) (10000,)
(5000, 784) (55000, 784) (5000,)


## Regular GD

Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 20 steps.

Performance evaluation: 

    time consumption: 89.35848474502563
    performance score: 0.9733

In [4]:
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10

In [5]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [6]:
with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')
    y_proba = tf.nn.softmax (y_predit, name = "y_predit")

In [7]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [11]:
import time

In [12]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_data[start: end], y: y_data[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_data[start: end], y: y_data[start: end]})
        validation_score = accuracy.eval (feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  0.9583333 the validation score is  0.9426 this cost 4.639434814453125
At step  1 the train_score is  1.0 the validation score is  0.955 this cost 4.4254233837127686
At step  2 the train_score is  1.0 the validation score is  0.9626 this cost 4.510042190551758
At step  3 the train_score is  1.0 the validation score is  0.9692 this cost 4.620443105697632
At step  4 the train_score is  1.0 the validation score is  0.972 this cost 4.435945272445679
At step  5 the train_score is  1.0 the validation score is  0.9748 this cost 4.447862863540649
At step  6 the train_score is  1.0 the validation score is  0.977 this cost 4.673173427581787
At step  7 the train_score is  1.0 the validation score is  0.9802 this cost 4.44305944442749
At step  8 the train_score is  1.0 the validation score is  0.9812 this cost 4.438606023788452
At step  9 the train_score is  1.0 the validation score is  0.9822 this cost 4.436694383621216
At step  10 the train_score is  1.0 the validat

In [40]:
reset_graph ()
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10

## Adam:
Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 30 steps.

Performance evaluation:

    time consumption:  114.90625596046448
    performance score:   0.1998

Appearently there is an error in decent, it seems moved too fast and missed the lowest point, we shall introduce the earily stop tech.
Idealy, at step 15 , you should stop the process. The train score dropped to  0.45833334 at step 15 same as validation score

Change the learning rate to 0.001 could resolve this issue, although I still don't understand why 

In [41]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [42]:
with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')

In [43]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [44]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [45]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [46]:
init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [47]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_train, train_score = sess.run ([loss, accuracy], feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
        print (loss_train)
    total_end_time = time.time ()
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  0.9583333 the validation score is  0.885 this cost 5.797502040863037
0.24410363
At step  1 the train_score is  0.20833333 the validation score is  0.1002 this cost 5.729186773300171
2.326124
At step  2 the train_score is  0.125 the validation score is  0.0986 this cost 5.7704548835754395
2.3226068
At step  3 the train_score is  0.20833333 the validation score is  0.1002 this cost 5.738136529922485
2.3200274
At step  4 the train_score is  0.20833333 the validation score is  0.1002 this cost 5.782637119293213
2.3207953
At step  5 the train_score is  0.20833333 the validation score is  0.1002 this cost 5.783485651016235
2.3212085
At step  6 the train_score is  0.20833333 the validation score is  0.1002 this cost 5.779216289520264
2.3212917
At step  7 the train_score is  0.20833333 the validation score is  0.1002 this cost 5.763638257980347
2.321304
At step  8 the train_score is  0.20833333 the validation score is  0.1002 this cost 5.767236948013306
2.3213055

## Adam with early stop:

Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 20 steps. earily stop if the loss not been reduced further for 35% of the steps.(at step 2, it seems randomly find a good result so stop earily)

Performance evaluation:

    time consumption: 51.90262746810913
    performance score: 0.9
    
Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 20 steps. earily stop if the loss not been reduced further for 30% of the steps.

Performance evaluation:

    time consumption: 61.913670778274536
    performance score: 0.8656



In [48]:
reset_graph ()
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    best_loss = 1000 
    without_update_counter = 0
    max_without_update = int (0.35 * n_steps)
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
        print ('At step ', step, 'the loss is ', loss_val, 'the best loss is ', best_loss, 'this cost', run_time)
        if loss_val < best_loss:
            save_path = saver.save (sess, r'C:\Users\asiaynrf\Project\model_saver\my_model.ckpt')
            best_loss = loss_val
            without_update_counter = 0
        else:
            without_update_counter += 1
            if without_update_counter > max_without_update:
                print ('earily stopped')
                break
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)

with tf.Session() as sess:
    saver.restore (sess, r'C:\Users\asiaynrf\Project\model_saver\my_model.ckpt')
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.8672 this cost 6.42667293548584
At step  0 the loss is  0.5876669 the best loss is  1000 this cost 6.42667293548584
At step  1 the train_score is  1.0 the validation score is  0.9408 this cost 6.17248010635376
At step  1 the loss is  0.26098397 the best loss is  0.5876669 this cost 6.17248010635376
At step  2 the train_score is  1.0 the validation score is  0.913 this cost 6.594958543777466
At step  2 the loss is  0.45288447 the best loss is  0.26098397 this cost 6.594958543777466
At step  3 the train_score is  1.0 the validation score is  0.9216 this cost 6.328110218048096
At step  3 the loss is  0.35236454 the best loss is  0.26098397 this cost 6.328110218048096
At step  4 the train_score is  1.0 the validation score is  0.9186 this cost 6.404296398162842
At step  4 the loss is  0.44964537 the best loss is  0.26098397 this cost 6.404296398162842
At step  5 the train_score is  1.0 the validation score is  0.9086 this cost 6

## GD without mini-batch
Performance evaluation: 

Score: (fogot to run the test set) 2000 steps's validation score is  0.9448. For the previous setup, it is 32 batch size, in another word, one step is an 1719 steps, so 20 step is 34380 steps in total. No wonder why mini-batch is better.

Take look at the time consumption, it is 1468s, compare to 120s in previous method to have similar (actually worse) performance. 

time consume: 1483.5600037574768

In [24]:
# reset_graph()

# X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
# y = tf.placeholder (tf.int32, shape = (None), name = 'y')

# with tf.name_scope ('DNN_chapter11_8'):
#     he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
#     hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
#     hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
#     hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
#     hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
#     hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
#     y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')

# with tf.name_scope ('loss'):
#     xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
#     loss = tf.reduce_mean(xentropy, name="loss")

# with tf.name_scope("train"):
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#     training_op = optimizer.minimize(loss)

# with tf.name_scope("eval"):
#     correct = tf.nn.in_top_k(y_predit, y, 1)
#     accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# init = tf.global_variables_initializer()
# saver = tf.train.Saver()
# n_steps = 2000
# batch_size = 32
# data_size = X_train.shape[0]
# n_batch = int (np.ceil (data_size / batch_size))

# with tf.Session() as sess:
#     init.run()
#     total_start_time = time.time()
#     best_loss = 1000 
#     without_update_counter = 0
#     max_without_update = int (0.2 * n_steps)
#     for step in range (n_steps):
#         start_time = time.time ()
#         sess.run (training_op, feed_dict = {X: X_train, y: y_train})
#         train_score = accuracy.eval (feed_dict = {X: X_train, y: y_train})
#         loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
#         end_time = time.time ()
#         run_time = end_time - start_time
#         print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
#         # print ('At step ', step, 'the loss is ', loss_val, 'the best loss is ', best_loss, 'this cost', run_time)
# #         if loss_val < best_loss:
# #             save_path = saver.save (sess, r'C:\Users\asiaynrf\Project\model_saver\my_model.ckpt')
# #             best_loss = loss_val
# #             without_update_counter = 0
# #         else:
# #             without_update_counter += 1
# #             if without_update_counter > max_without_update:
# #                 print ('earily stopped')
# #                 break
#     total_end_time = time.time ()
#     print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
#     accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
#     print ('the test set accuracy is', accuracy_val)

## Add Batch Normalization

    run time 160.15829229354858
    score :  0.9071
if take a look, the converge is geting much slower

In [25]:
reset_graph ()

In [26]:
from functools import partial

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    training = tf.placeholder_with_default (False, shape = (), name = 'training')
    Batch_norm_layer = partial (tf.layers.batch_normalization, training = training, momentum = 0.9)
    hidden1 = tf.layers.dense (X, n_neurons, kernel_initializer = he_init, name = 'Hidden1')
    batch_h1 = Batch_norm_layer (hidden1)
    batch_h1_act = tf.nn.elu (batch_h1)
    hidden2 = tf.layers.dense (batch_h1_act, n_neurons, kernel_initializer = he_init, name = 'Hidden2')
    batch_h2 = Batch_norm_layer (hidden2)
    batch_h2_act = tf.nn.elu (batch_h2)
    hidden3 = tf.layers.dense (batch_h2_act, n_neurons, kernel_initializer = he_init, name = 'Hidden3')
    batch_h3 = Batch_norm_layer (hidden3)
    batch_h3_act = tf.nn.elu (batch_h3)
    hidden4 = tf.layers.dense (batch_h3_act, n_neurons, kernel_initializer = he_init, name = 'Hidden4')
    batch_h4 = Batch_norm_layer (hidden4)
    batch_h4_act = tf.nn.elu (batch_h4)
    hidden5 = tf.layers.dense (batch_h4_act, n_neurons, kernel_initializer = he_init, name = 'Hidden5')
    batch_h5 = Batch_norm_layer (hidden5)
    batch_h5_act = tf.nn.elu (batch_h5)
    y_predit_before_norm = tf.layers.dense (batch_h5_act, n_output, kernel_initializer = he_init, name = 'Output')
    y_predit = Batch_norm_layer (y_predit_before_norm)

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
extra_update_ops = tf.get_collection (tf.GraphKeys.UPDATE_OPS)
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run ([training_op, training], feed_dict = {training: True , X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  0.7083333 the validation score is  0.6438 this cost 8.860485792160034
At step  1 the train_score is  0.7083333 the validation score is  0.6846 this cost 8.325199127197266
At step  2 the train_score is  0.7916667 the validation score is  0.7202 this cost 8.837181806564331
At step  3 the train_score is  0.8333333 the validation score is  0.7434 this cost 8.532508373260498
At step  4 the train_score is  0.875 the validation score is  0.7686 this cost 8.110628366470337
At step  5 the train_score is  0.9166667 the validation score is  0.7892 this cost 8.592307329177856
At step  6 the train_score is  0.9166667 the validation score is  0.8064 this cost 8.850103616714478
At step  7 the train_score is  0.9166667 the validation score is  0.8194 this cost 9.84930157661438
At step  8 the train_score is  0.9166667 the validation score is  0.828 this cost 9.105423927307129
At step  9 the train_score is  0.9166667 the validation score is  0.8334 this cost 9.221487522125

## Add gradiant clip

 The gradiant clip applied at training op not inside of DNN
 
     Performance: 0.9735
     Time comume: 89.36819243431091

In [27]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads_and_vars  = optimizer.compute_gradients (loss)
    capped_grads_and_vars = [(tf.clip_by_value(g,-clip_threshold, clip_threshold), v) for g,v in grads_and_vars]
    training_op = optimizer.apply_gradients (capped_grads_and_vars)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
extra_update_ops = tf.get_collection (tf.GraphKeys.UPDATE_OPS)
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [29]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9356 this cost 4.768087387084961
At step  1 the train_score is  1.0 the validation score is  0.952 this cost 4.486506223678589
At step  2 the train_score is  1.0 the validation score is  0.9586 this cost 4.470193147659302
At step  3 the train_score is  1.0 the validation score is  0.9634 this cost 4.464956045150757
At step  4 the train_score is  1.0 the validation score is  0.9674 this cost 4.529657602310181
At step  5 the train_score is  1.0 the validation score is  0.969 this cost 4.511454343795776
At step  6 the train_score is  1.0 the validation score is  0.9698 this cost 4.470094442367554
At step  7 the train_score is  1.0 the validation score is  0.97 this cost 4.475607633590698
At step  8 the train_score is  1.0 the validation score is  0.9704 this cost 4.454093933105469
At step  9 the train_score is  1.0 the validation score is  0.9706 this cost 4.4885993003845215
At step  10 the train_score is  1.0 the validation sc

## Add dropout

In [30]:
reset_graph ()

In [31]:
import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
dropout_rate = 0.5

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')
training = tf.placeholder_with_default (False, shape = (), name = 'Training_indicator')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    X_dropped = tf.layers.dropout (X, rate = dropout_rate, training = training)
    inputs = X_dropped
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
        inputs = tf.layers.dropout (inputs, rate = dropout_rate, training = training)
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads_and_vars  = optimizer.compute_gradients (loss)
    capped_grads_and_vars = [(tf.clip_by_value(g,-clip_threshold, clip_threshold), v) for g,v in grads_and_vars]
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
extra_update_ops = tf.get_collection (tf.GraphKeys.UPDATE_OPS)
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [32]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {training: True, X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)
    writer = tf.summary.FileWriter(r"C:\Users\asiaynrf\Desktop\fun\handson_ML\Graph_saver", sess.graph)

At step  0 the train_score is  0.6666667 the validation score is  0.6528 this cost 5.542396783828735
At step  1 the train_score is  0.875 the validation score is  0.7806 this cost 5.366839647293091
At step  2 the train_score is  0.875 the validation score is  0.7954 this cost 5.343973875045776
At step  3 the train_score is  0.875 the validation score is  0.8064 this cost 5.36245322227478
At step  4 the train_score is  0.9583333 the validation score is  0.8488 this cost 5.3312554359436035
At step  5 the train_score is  0.9583333 the validation score is  0.861 this cost 5.351331472396851
At step  6 the train_score is  0.9583333 the validation score is  0.8724 this cost 5.349643230438232
At step  7 the train_score is  0.9583333 the validation score is  0.852 this cost 5.3778181076049805
At step  8 the train_score is  0.9583333 the validation score is  0.8708 this cost 5.343310832977295
At step  9 the train_score is  0.9583333 the validation score is  0.8844 this cost 5.374162435531616
At 

## Momentum with momentum = 0.9

In [33]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
momentum_value = 0.9

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = momentum_value)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [34]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9484 this cost 5.273808002471924
At step  1 the train_score is  1.0 the validation score is  0.9644 this cost 5.7890400886535645
At step  2 the train_score is  1.0 the validation score is  0.9658 this cost 4.836617708206177
At step  3 the train_score is  1.0 the validation score is  0.9684 this cost 4.849977493286133
At step  4 the train_score is  1.0 the validation score is  0.965 this cost 4.846926689147949
At step  5 the train_score is  1.0 the validation score is  0.9696 this cost 4.839024305343628
At step  6 the train_score is  1.0 the validation score is  0.97 this cost 4.955053329467773
At step  7 the train_score is  1.0 the validation score is  0.9776 this cost 4.829431056976318
At step  8 the train_score is  1.0 the validation score is  0.9764 this cost 4.834311008453369
At step  9 the train_score is  1.0 the validation score is  0.978 this cost 4.828329086303711
At step  10 the train_score is  1.0 the validation sc

## Optimizer: AdaGrad

In [35]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
momentum_value = 0.9

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.AdagradOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [36]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9526 this cost 5.174942255020142
At step  1 the train_score is  1.0 the validation score is  0.9628 this cost 5.226587772369385
At step  2 the train_score is  1.0 the validation score is  0.968 this cost 5.039770126342773
At step  3 the train_score is  1.0 the validation score is  0.9698 this cost 4.967498540878296
At step  4 the train_score is  1.0 the validation score is  0.972 this cost 5.003436803817749
At step  5 the train_score is  1.0 the validation score is  0.9734 this cost 5.06840968132019
At step  6 the train_score is  1.0 the validation score is  0.9746 this cost 5.402022123336792
At step  7 the train_score is  1.0 the validation score is  0.975 this cost 5.40156888961792
At step  8 the train_score is  1.0 the validation score is  0.9756 this cost 5.0783350467681885
At step  9 the train_score is  1.0 the validation score is  0.9758 this cost 4.975041627883911
At step  10 the train_score is  1.0 the validation sco

## Optimizer: RMSProp

In [37]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
momentum_value = 0.9

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.RMSPropOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [38]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9314 this cost 5.789090633392334
At step  1 the train_score is  1.0 the validation score is  0.9376 this cost 5.8565919399261475
At step  2 the train_score is  1.0 the validation score is  0.9268 this cost 5.766861438751221
At step  3 the train_score is  1.0 the validation score is  0.926 this cost 5.696022033691406
At step  4 the train_score is  1.0 the validation score is  0.9306 this cost 5.832495450973511
At step  5 the train_score is  1.0 the validation score is  0.927 this cost 5.9576849937438965
At step  6 the train_score is  1.0 the validation score is  0.9404 this cost 5.83569073677063
At step  7 the train_score is  1.0 the validation score is  0.9274 this cost 5.798902273178101
At step  8 the train_score is  0.9583333 the validation score is  0.8946 this cost 5.5825793743133545
At step  9 the train_score is  1.0 the validation score is  0.9248 this cost 5.615818977355957
At step  10 the train_score is  1.0 the vali

## Optimizer: AdadeltaOptimizer

In [49]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
momentum_value = 0.9

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.AdadeltaOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [50]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  0.6666667 the validation score is  0.683 this cost 6.392651081085205
At step  1 the train_score is  0.875 the validation score is  0.8058 this cost 6.126388788223267
At step  2 the train_score is  1.0 the validation score is  0.8464 this cost 6.125507831573486
At step  3 the train_score is  1.0 the validation score is  0.8686 this cost 6.352351427078247
At step  4 the train_score is  1.0 the validation score is  0.8794 this cost 6.359858989715576
At step  5 the train_score is  1.0 the validation score is  0.8918 this cost 6.308007001876831
At step  6 the train_score is  1.0 the validation score is  0.8988 this cost 6.431229829788208
At step  7 the train_score is  1.0 the validation score is  0.9032 this cost 6.345709800720215
At step  8 the train_score is  1.0 the validation score is  0.907 this cost 6.239832878112793
At step  9 the train_score is  1.0 the validation score is  0.9108 this cost 6.468474388122559
At step  10 the train_score is  1.0 the vali

## FtrlOptimizer

In [52]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
momentum_value = 0.9

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.FtrlOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [53]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  0.125 the validation score is  0.1126 this cost 6.807048797607422
At step  1 the train_score is  0.125 the validation score is  0.1126 this cost 6.715728282928467
At step  2 the train_score is  0.125 the validation score is  0.1126 this cost 6.106866359710693
At step  3 the train_score is  0.125 the validation score is  0.1126 this cost 5.94976806640625
At step  4 the train_score is  0.125 the validation score is  0.1126 this cost 5.927315950393677
At step  5 the train_score is  0.125 the validation score is  0.1126 this cost 5.909650802612305
At step  6 the train_score is  0.125 the validation score is  0.1126 this cost 5.941744565963745
At step  7 the train_score is  0.125 the validation score is  0.1126 this cost 5.891998529434204
At step  8 the train_score is  0.125 the validation score is  0.1126 this cost 5.908383131027222
At step  9 the train_score is  0.125 the validation score is  0.1126 this cost 5.899424314498901
At step  10 the train_score is 

## ProximalGradientDescentOptimizer

In [55]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
momentum_value = 0.9

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9356 this cost 5.803193092346191
At step  1 the train_score is  1.0 the validation score is  0.952 this cost 5.648366928100586
At step  2 the train_score is  1.0 the validation score is  0.9586 this cost 5.634929180145264
At step  3 the train_score is  1.0 the validation score is  0.9634 this cost 5.813776016235352
At step  4 the train_score is  1.0 the validation score is  0.9674 this cost 5.643099784851074
At step  5 the train_score is  1.0 the validation score is  0.969 this cost 5.659044027328491
At step  6 the train_score is  1.0 the validation score is  0.9698 this cost 5.649252653121948
At step  7 the train_score is  1.0 the validation score is  0.97 this cost 5.6877360343933105
At step  8 the train_score is  1.0 the validation score is  0.9704 this cost 5.67298150062561
At step  9 the train_score is  1.0 the validation score is  0.9706 this cost 5.6518800258636475
At step  10 the train_score is  1.0 the validation sc

## ProximalAdagradOptimizer

In [56]:
reset_graph ()

import time
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10
clip_threshold = 1
momentum_value = 0.9

X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    inputs = X
    for layer in range (n_layers):
        inputs = tf.layers.dense (inputs, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name="hidden%d" % (layer + 1))
    y_predit = tf.layers.dense (inputs, n_output, kernel_initializer = he_init, name = 'output')
    y_proba = tf.nn.softmax(y_predit, name="Y_proba")

with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.ProximalAdagradOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9526 this cost 5.705142021179199
At step  1 the train_score is  1.0 the validation score is  0.9628 this cost 5.7057108879089355
At step  2 the train_score is  1.0 the validation score is  0.968 this cost 5.620474815368652
At step  3 the train_score is  1.0 the validation score is  0.9698 this cost 5.634216070175171
At step  4 the train_score is  1.0 the validation score is  0.972 this cost 5.629890441894531
At step  5 the train_score is  1.0 the validation score is  0.9734 this cost 5.632022857666016
At step  6 the train_score is  1.0 the validation score is  0.9746 this cost 5.684013605117798
At step  7 the train_score is  1.0 the validation score is  0.975 this cost 5.673597812652588
At step  8 the train_score is  1.0 the validation score is  0.9756 this cost 5.906827449798584
At step  9 the train_score is  1.0 the validation score is  0.9758 this cost 5.939324140548706
At step  10 the train_score is  1.0 the validation s